In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import requests

## Data Collection

### Files sources

In [ ]:
# URLs of the files
train_datas_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/module4_exercise_train.zip'
test_data_url = 'https://www.raphaelcousin.com/modules/data-science-practice/module4/exercise/Neighborhood_Market_data.csv'

# Function to download a file
def download_file(url, file_name):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we notice bad responses
    with open(file_name, 'wb') as file:
        file.write(response.content)
    print(f'Downloaded {file_name} from {url}')

# Downloading the files
download_file(train_datas_url, 'module4_exercise_train.zip')
download_file(test_data_url, 'Neighborhood_Market_data.csv')

#### CityMart

In [ ]:
# read "CityMart_data.csv"
citymart_pd = pd.read_csv('./module4_exercise_train/CityMart_data.csv')

citymart_pd.head()

#### Greenfield_Grocers

In [ ]:
# read "Greenfield_Grocers_data.csv"
greenfield_grocers_pd = pd.read_csv('./module4_exercise_train/Greenfield_Grocers_data.csv', 
                                    delimiter='|',
                                    header=3
)

greenfield_grocers_pd.head()

#### Outlet_data

In [ ]:
# read "SuperSaver_Outlet_data.xlsx"
supersaver_outlet_pd = pd.read_excel('./module4_exercise_train/SuperSaver_Outlet_data.xlsx')

supersaver_outlet_pd.head()


#### HighStreet_Bazaar

In [ ]:
# read 'HighStreet_Bazaar_data.json'
highstreet_bazaar_pd = pd.read_json('./module4_exercise_train/HighStreet_Bazaar_data.json')
highstreet_bazaar_pd.head()

#### Aggregate

In [ ]:
concat_df = pd.concat(
    [citymart_pd, greenfield_grocers_pd, supersaver_outlet_pd, highstreet_bazaar_pd], 
    axis=0, 
    ignore_index=True
)
concat_df

#### Simple baseline

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np

def get_simple_baseline(data, fillna_value=-1, drop_cols=None, k_fold=5, scaler='standard', model='linear', metric='mae', target_col=None, X_data_test=None):
    
    data = data.copy()
    # Handle missing values
    data.fillna(fillna_value, inplace=True)
    if X_data_test is not None:
        X_data_test = X_data_test.copy()
        X_data_test.fillna(fillna_value, inplace=True)
    
    # Drop unwanted columns
    if drop_cols:
        data.drop(drop_cols, axis=1, inplace=True)
        if X_data_test is not None:
            X_data_test.drop(drop_cols, axis=1, inplace=True)

    # Split data into features (X) and target (y)
    y = data[target_col]
    X = data.drop(target_col, axis=1)

    # Feature scaling
    if scaler == 'standard':
        scaler = StandardScaler()
    elif scaler == 'minmax':
        scaler = MinMaxScaler()
    else:
        scaler = None
    
    if scaler:
        X = scaler.fit_transform(X)
        if X_data_test is not None:
            X_data_test = scaler.transform(X_data_test)

    # Initialize the model
    if model == 'linear':
        model = LinearRegression()
    elif model == 'logistic':
        model = LogisticRegression()
    elif model == 'random_forest':
        model = RandomForestClassifier()
    else:
        raise ValueError("Unsupported model type")

    # Initialize cross-validation
    kf = KFold(n_splits=k_fold, shuffle=True, random_state=42)
    scores = []

    # Train and evaluate using k-fold cross-validation
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate using the specified metric
        if metric == 'mae':
            score = mean_absolute_error(y_test, y_pred)
        elif metric == 'accuracy':
            score = accuracy_score(y_test, np.round(y_pred))
        else:
            raise ValueError("Unsupported metric")

        scores.append(score)

    if X_data_test is not None:
        model.fit(X, y)
        return np.mean(scores), model.predict(X_data_test)
    
    # Return the average score
    return np.mean(scores)

In [ ]:
data = concat_df.copy()
bad_cols = [
    'ITEM_CODE', 'STORE_NAME', 'LAST_MODIFIED', 'QUANTITY_SOLD',    # doublons en MAJ
    'Unnamed: 12', '1' 
]
# data.drop(columns=bad_cols, errors='ignore')
data = data.drop(columns=[c for c in bad_cols if c in data.columns], errors='ignore')

# score1, y_pred1 = get_simple_baseline(data, fillna_value=-1, drop_cols=['item_code', 'store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold', X_data_test=data.drop('quantity_sold', axis=1))
score1 = get_simple_baseline(data, fillna_value=-1, drop_cols=['item_code', 'store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')
score1

In [ ]:
data

### API sources

In [ ]:
def get_api(endpoint_url):
    try:
        # Make the GET request to the mock API
        response = requests.get(endpoint_url)

        if response.status_code == 200:
            data = response.json()
            print(data["message"])
            return data['data']
        else:
            print(f"Failed to retrieve volume data. Status code: {response.status_code}")
    
    except Exception as e:
        print(f"An error occurred: {e}")
# password = 
# prices = 

In [ ]:
get_api('https://www.raphaelcousin.com/api/exercise/auth')

In [ ]:
password = 'RcUZjhdsYLRzwi4'
endpoint_url = f'https://www.raphaelcousin.com/api/exercise/{password}/prices'
prices = get_api(endpoint_url)

#### Aggregate

In [ ]:
columns = ['item_code', 'prices']
df_prices = pd.DataFrame(list(prices.items()), columns=columns)
df_prices

In [ ]:
data = pd.merge(data, df_prices, on='item_code', how='left')
data

In [ ]:
score2 = get_simple_baseline(data, fillna_value=-1, drop_cols=['item_code', 'store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

if score2 < score1:
    print("Amélioration du modèle !")
else:
    print("Modèle moins bien...")
score2

### Scrapping sources

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver (e.g., Chrome)
driver = webdriver.Chrome()  # Make sure ChromeDriver is installed
# driver = webdriver.Firefox()
# driver = webdriver.Edge()
# driver = webdriver.Safari()

# Open the URL
url = 'https://www.raphaelcousin.com/module4/scrapable-data'
driver.get(url)

# Wait for the page to fully load (increase time if needed)
time.sleep(5)

# Get the fully rendered page source
html = driver.page_source

# Parse the HTML with BeautifulSoup
soup = BeautifulSoup(html, 'html.parser')

# Initialize lists to store scraped data
exercise_data = []

# Find both tables
tables = soup.find_all('table')

# Close the Selenium WebDriver
driver.quit()

# Scrape the second table (Exercise Data)
course_table = tables[1]
for row in course_table.find('tbody').find_all('tr'):
    cols = row.find_all('td')
    exercise_data.append([col.get_text() for col in cols])

# Convert the lists to pandas DataFrames
df_exercise = pd.DataFrame(exercise_data, columns=['item_code', 'customer_score', 'total_reviews', 'updated_timestamp'])
df_exercise


#### Aggregate

In [ ]:
data

In [ ]:
data = pd.merge(data, df_exercise, on='item_code', how='left')
data

In [ ]:
score3 = get_simple_baseline(data, fillna_value=-1, drop_cols=['item_code', 'store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold')

if score3 < score2:
    print("Amélioration du modèle !")
elif score3 < score1:
    print("Moins bien que mmodèle 2, mais mieux que modèle 1.")
else:
    print("Modèle moins bien...")
score3

### Generating Submission File

In [ ]:
# X_test =  read  Neighborhood_Market_data

# read
df_StoreN =  pd.read_csv("Neighborhood_Market_data.csv", sep=",", index_col='item_code')
df_StoreN

In [ ]:
data.columns = data.columns.str.lower()
df_StoreN = pd.merge(df_StoreN, df_prices, on='item_code', how='left')
df_StoreN = pd.merge(df_StoreN, df_exercise, on='item_code', how='left')
df_StoreN = df_StoreN[data.columns.drop('quantity_sold')]
df_StoreN

In [ ]:
score_test, x_pred = get_simple_baseline(data, fillna_value=-1, drop_cols=['item_code', 'store_name', 'last_modified'], k_fold=5, scaler='standard', model='linear', metric='mae', target_col='quantity_sold', X_data_test = df_StoreN)
score_test

In [ ]:
df_StoreN.index

In [ ]:
x_pred

In [ ]:
submission = pd.DataFrame({
    'item_code': df_StoreN.index,
    'quantity_sold': x_pred # your_prediction
})

submission.to_csv('submission.csv', index=False, sep=',')
submission.head()